## Newton para Sistemas não Lineares

O método de Newton para sistemas não lineares busca encontrar a solução de um sistema f(x) = 0 utilizando aproximações sucessivas e a inversa do Jacobiano.


In [ ]:
import numpy as np
import sympy as sp
from tabulate import tabulate

In [2]:
def newton_sistemas(funcs, vars, x0, tol, max_iter=100):
    """
    Resolve sistemas de equações não lineares pelo método de Newton.

    Parâmetros:
    funcs -- Lista de funções do sistema (expressões simbólicas)
    vars -- Lista de variáveis simbólicas
    x0 -- Aproximação inicial (array ou lista)
    tol -- Erro de tolerância
    max_iter -- Número máximo de iterações

    Retorna:
    Solução aproximada e tabela com as iterações
    """
    # Calcula o Jacobiano simbólico
    J = sp.Matrix([[sp.diff(f, var) for var in vars] for f in funcs])
    
    # Converte as funções e o Jacobiano para funções numéricas
    f_lambdify = sp.lambdify(vars, funcs, 'numpy')
    J_lambdify = sp.lambdify(vars, J, 'numpy')

    # Inicializa a solução inicial
    x = np.array(x0, dtype=float)
    tabela = []

    for iter in range(max_iter):
        # Avalia as funções e o Jacobiano no ponto atual
        F_val = np.array(f_lambdify(*x), dtype=float).flatten()
        J_val = np.array(J_lambdify(*x), dtype=float)

        # Verifica se o Jacobiano está mal condicionado
        if np.linalg.cond(J_val) > 1e12:
            print("Jacobiano mal condicionado! Pode não haver convergência.")
            return None, tabela

        try:
            # Resolve o sistema linear J * delta_x = -F
            delta_x = np.linalg.solve(J_val, -F_val)
        except np.linalg.LinAlgError:
            print("Erro: Jacobiano singular ou indefinido.")
            return None, tabela

        # Atualiza a solução
        x_new = x + delta_x

        # Calcula os erros absoluto e relativo
        erro_absoluto = np.linalg.norm(delta_x)
        erro_relativo = erro_absoluto / (np.linalg.norm(x_new) if np.linalg.norm(x_new) != 0 else 1)

        # Armazena os resultados na tabela
        tabela.append([iter, *x, erro_absoluto, erro_relativo])

        # Verifica a condição de parada
        if erro_absoluto <= tol:
            return np.round(x_new, 4), tabela

        # Atualiza o ponto para a próxima iteração
        x = x_new

    print("Número máximo de iterações atingido.")
    return np.round(x, 4), tabela

In [4]:
funcs = [sp.sympify("10*x1-(25*cos(x1))-10*x2+10*x1"), sp.sympify("10*x2-10*x1-(15*cos(x2))-10*x3+10*x2"), sp.sympify("10*x3-10*x2-(5*cos(x3))")]

In [7]:
# Entrada do usuário
if __name__ == "__main__":
    n = int(input("Digite o número de equações/variáveis: "))
    vars = sp.symbols(f'x1:{n+1}')
    
    # Define as funções do sistema
    # print("Digite as funções do sistema:")
    # funcs = []
    # for i in range(n):
    #     func_input = input(f"f{i+1}(x1, ..., xn) = ")
    #     funcs.append(sp.sympify(func_input))

    # Define a aproximação inicial
    x0 = [float(input(f"Digite x{i+1} inicial: ")) for i in range(n)]
    tol = float(input("Digite o erro de tolerância: "))

    # Executa o método de Newton
    resultado = newton_sistemas(funcs, vars, x0, tol)

    if resultado is None:
        print("O método não convergiu ou houve um erro numérico.")
    else:
        solucao, tabela = resultado
        
        # Exibe a tabela de resultados
        headers = ["Iteração"] + [f"x{i+1}" for i in range(n)] + ["Erro Absoluto", "Erro Relativo"]
        print(tabulate(tabela, headers=headers, floatfmt=".4f"))
        
        print(f"Solução aproximada: {solucao}")

Número máximo de iterações atingido.
  Iteração                   x1                   x2                   x3       Erro Absoluto    Erro Relativo
----------  -------------------  -------------------  -------------------  ------------------  ---------------
         0               0.0000               0.0000               0.0000             10.5594           1.0000
         1               4.5000               6.5000               7.0000              3.5828           0.2598
         2               7.3760               8.2505               8.2251              2.1269           0.1810
         3               5.8334               7.0925               7.3288              5.2485           0.6587
         4               1.1524               5.1414               5.9769              4.0731           0.3406
         5               2.6698               7.7903               8.6735              3.2617           0.3747
         6               2.0856               5.7343               6.2097  